In [88]:
import json
import pandas as pd
import copy
import re
import itertools as it
from collections import Counter

In [2]:
from nltk.corpus import stopwords
stop_words=stopwords.words("english")
saved_copy=pd.read_json("tweets/2020.json")

In [41]:
del w20

In [42]:
w20=copy.copy(saved_copy)

In [43]:
pd.set_option("display.max_colwidth", 500)
pd.set_option("display.min_rows", 20)

In [44]:
w20=w20.drop(["source"], axis=1)

In [45]:
w20.insert(loc=5, column="is_retweet_fix", value=[True if re.search(pattern="RT",string=i) else False for i in w20.text])

In [46]:
#w20.insert(loc=2, column="year_c", value=pd.to_datetime(w20.created_at).dt.year)
#w20.insert(loc=2, column="month_c", value=pd.to_datetime(w20.created_at).dt.month)
#w20.insert(loc=2, column="day_c", value=pd.to_datetime(w20.created_at).dt.day)

In [47]:
w20["text"]=[re.sub(".@"," @",i) for i in w20.text]

In [48]:
w20.insert(loc=0, column="text_1", value=[re.sub(pattern=r"https://[\w./]*",repl="",string=i) for i in w20.text])
w20.insert(loc=0, column="text_2", value=[re.sub(pattern=r"RT[@|\w|\s]*:",repl="",string=i) for i in w20.text_1])
w20.insert(loc=0, column="text_3", value=[re.split(pattern=r"[\s|\.]",string=i) for i in w20.text_2])

w20.insert(loc=0, column="mentions", value=list(map(lambda line: list(filter(lambda x: x.startswith("@"), line)), w20.text_3)))
w20.insert(loc=0, column="hashtags", value=list(map(lambda line: list(filter(lambda x: x.startswith("#"), line)), w20.text_3)))
w20.insert(loc=0, column="yelling", value=list(map(lambda line: list(filter(lambda x: x.isupper(), line)), w20.text_3)))

In [97]:
ord("’")

8217

In [113]:
r"|".join([chr(i) for i in range(8216,8224)])

'‘|’|‚|‛|“|”|„|‟'

In [49]:
w20.insert(loc=0, column="text_4", value=[" ".join(i) for i in w20.text_3])

In [116]:
clean_up=r"—|…|_|-|;|&amp;|%|\?|!|\.|\,|\)|\(\"|\'|’"+chr(12540)+"|"+r"|".join([chr(i) for i in range(8216,8224)])

In [115]:
w20.insert(loc=0, column="text_5", value=[re.sub(clean_up, "", i) for i in w20.text_4])

ValueError: cannot insert text_5, already exists

In [51]:
covid_patterns=re.compile(r"[c|C][o|O][v|V][i|I][d|D][-_|\s]?[\d]*|[c|C]oronavirus")

In [52]:
w20.insert(loc=0, column="corona", value=[covid_patterns.findall(i) for i in w20.text_5])

In [53]:
w20.insert(loc=0, column="text_6", value=[i.split() for i in w20.text_5])

In [118]:
new_stop_words = stop_words + ["great", "president", "thank","people","wall","country","big","new","time","much","many","year","want","need","via"]
new_stop_words = new_stop_words + [i.capitalize() for i in new_stop_words]

In [119]:
w20=w20.drop(["text_7"], axis=1)

In [120]:
w20.insert(loc=0, column="text_7", value=list(map(lambda line: list(filter(lambda x: x not in new_stop_words, line)), w20.text_6)))

In [129]:
from nltk.util import bigrams, trigrams

In [130]:
g=[[" ".join(i) for i in (list(trigrams(line)))] for line in w20.text_7]

In [131]:
newg=list(it.chain.from_iterable(g))

In [132]:
g_counter=Counter(newg)

In [133]:
g_counter.most_common()

[('Complete Total Endorsement', 132),
 ('Fake News Media', 38),
 ('Approval Rating Republican', 35),
 ('Rating Republican Party', 35),
 ('Sleepy Joe Biden', 34),
 ('Coronavirus Task Force', 31),
 ('LIVE: Press Briefing', 29),
 ('Press Briefing Coronavirus', 29),
 ('Briefing Coronavirus Task', 29),
 ('AMERICA GREAT AGAIN', 28),
 ('MAKE AMERICA GREAT', 27),
 ('LIVE: @realDonaldTrump holds', 25),
 ('Radical Left Nothing', 25),
 ('Left Nothing Democrats', 23),
 ('Mini Mike Bloomberg', 21),
 ('Fake News @CNN', 20),
 ('@realDonaldTrump holds news', 20),
 ('holds news conference', 20),
 ('Paycheck Protection Program', 20),
 ('White House Press', 18),
 ('96 Approval Rating', 17),
 ('Radical Left Democrats', 17),
 ('Loves Military Vets', 17),
 ('supporter #MAGA Agenda', 16),
 ('95 Approval Rating', 16),
 ('KEEP AMERICA GREAT', 16),
 ('DRAIN THE SWAMP', 15),
 ('Corrupt Joe Biden', 14),
 ('Cryin’ Chuck Schumer', 14),
 ('Vets Second Amendment', 13),
 ('Border Second Amendment', 12),
 ('Military Ve

In [66]:
w20

,text_7,text_6,corona,text_5,text_4,yelling,hashtags,mentions,text_3,text_2,...,text,created_at,day_c,month_c,year_c,retweet_count,favorite_count,is_retweet,is_retweet_fix,id_str
0,"[#TrumpTrain, Put, #earlyvote, #Trump, #Florida4Trump, #GOP, #tcot, 🇺🇸🇺🇸, @DanScavino]","[On, the, #TrumpTrain, Put, our, #earlyvote, in, for, #Trump, #Florida4Trump, #GOP, #tcot, 🇺🇸🇺🇸, @DanScavino]",[],On the #TrumpTrain Put our #earlyvote in for #Trump #Florida4Trump #GOP #tcot 🇺🇸🇺🇸 @DanScavino,On the #TrumpTrain Put our #earlyvote in for #Trump #Florida4Trump #GOP #tcot 🇺🇸🇺🇸 @DanScavino,[#GOP],"[#TrumpTrain, #earlyvote, #Trump, #Florida4Trump, #GOP, #tcot]",[@DanScavino],"[, On, the, #TrumpTrain, Put, our, #earlyvote, in, for, #Trump, , #Florida4Trump, #GOP, #tcot, 🇺🇸🇺🇸, @DanScavino, ]",On the #TrumpTrain.Put our #earlyvote in for #Trump. #Florida4Trump #GOP #tcot 🇺🇸🇺🇸 @DanScavino,...,RT @skb_sara: On the #TrumpTrain.Put our #earlyvote in for #Trump. #Florida4Trump #GOP #tcot 🇺🇸🇺🇸 @DanScavino https://t.co/fzIRPddA1z,2020-08-19 04:59:21+00:00,19,8,2020,6982,0,True,True,1295948444146728960
1,"[Raise, hand, youll, voting, straight, Republican, 77, days, like]","[Raise, your, hand, if, youll, be, voting, straight, Republican, in, 77, days, like, me]",[],Raise your hand if youll be voting straight Republican in 77 days like me,Raise your hand if you'll be voting straight Republican in 77 days like me!,[],[],[],"[, Raise, your, hand, if, you'll, be, voting, straight, Republican, in, 77, days, like, me!]",Raise your hand if you'll be voting straight Republican in 77 days like me!,...,RT @MrMichaelBurkes: Raise your hand if you'll be voting straight Republican in 77 days like me!,2020-08-19 04:57:43+00:00,19,8,2020,17076,0,True,True,1295948035483148288
2,"[support, President, Trump, support, Patriots, matter]","[If, you, support, President, Trump, then, i, support, you, All, Patriots, matter]",[],If you support President Trump then i support you All Patriots matter,If you support President Trump then i support you All Patriots matter!,[],[],[],"[, If, you, support, President, Trump, then, i, support, you, , All, Patriots, matter!, ]",If you support President Trump then i support you. All Patriots matter!,...,RT @MrMichaelBurkes: If you support President Trump then i support you. All Patriots matter! https://t.co/pgw3NduLT1,2020-08-19 04:57:33+00:00,19,8,2020,12045,0,True,True,1295947993594699776
3,"[Senate, Intelligence, Panel, found, absolutely, evidence, collusion, evidence, Trump, campaign, conspired, Russian, government, influence, 2016, affirming, findings, investigations, Daily, Caller]","[Senate, Intelligence, Panel, found, absolutely, no, evidence, of, collusion, There, was, no, evidence, that, the, Trump, campaign, conspired, with, the, Russian, government, to, influence, 2016, affirming, the, findings, of, other, investigations, Daily, Caller]",[],Senate Intelligence Panel found absolutely no evidence of collusion There was no evidence that the Trump campaign conspired with the Russian government to influence 2016 affirming the findings of other investigations Daily Caller,"“Senate Intelligence Panel found absolutely no evidence of collusion There was no evidence that the Trump campaign conspired with the Russian government to influence 2016, affirming the findings of other investigations ” Daily Caller",[],[],[],"[“Senate, Intelligence, Panel, found, absolutely, no, evidence, of, collusion, , There, was, no, evidence, that, the, Trump, campaign, conspired, with, the, Russian, government, to, influence, 2016,, affirming, the, findings, of, other, investigations, ”, Daily, Caller]","“Senate Intelligence Panel found absolutely no evidence of collusion. There was no evidence that the Trump campaign conspired with the Russian government to influence 2016, affirming the findings of other investigations.” Daily Caller",...,"“Senate Intelligence Panel found absolutely no evidence of collusion. There was no evidence that the Trump campaign conspired w

In [ ]:
#!/usr/bin/env python
"""
Minimal Example
===============
Generating a square wordcloud from the US constitution using default arguments.
"""

import os

from os import path
from wordcloud import WordCloud

# get data directory (using getcwd() is needed to support running example in generated IPython notebook)
d = path.dirname(__file__) if "__file__" in locals() else os.getcwd()

# Read the whole text.
text = open(path.join(d, 'constitution.txt')).read()

# Generate a word cloud image
wordcloud = WordCloud().generate(text)

# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")

# lower max_font_size
wordcloud = WordCloud(max_font_size=40).generate(text)
plt.figure()
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.show()

# The pil way (if you don't have matplotlib)
# image = wordcloud.to_image()
# image.show()